In [1]:
import tensorflow as tf
from datetime import timedelta
import numpy as np
tf.__version__

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


'2.5.0-rc1'

## Preparing X, Y

In [18]:
print("Enter timestep : ",end=' ')
timesteps = int(input())
print(timesteps)

Enter timestep :  2


In [125]:
import pandas as pd
from collections import defaultdict

provinces = ['Bangkok','Chanthaburi','Chiang Mai','Kanchanaburi','Songkhla']
path = './data/Train/fire_integrated/'
data = {}

X = defaultdict(lambda: list())
Y = defaultdict(lambda: list())

hour_step = timedelta(hours=timesteps-1)

predict_at = pd.date_range(start="2016-03-04 00:00:00", end="2019-03-17 23:00:00", freq='6H')

for province in provinces:
    df = pd.read_csv(path+f'{province}_fire_integrated.csv', index_col=0, parse_dates=True)
    data[province] = df

    for base in predict_at:

        if base-hour_step not in df.index : continue
        x = df.loc[base-hour_step: base].drop(['PM2.5'], axis=1)

        till = base+timedelta(hours=72)
        if till not in df.index: break
        y = df.loc[base+timedelta(hours=1): till, ['PM2.5']]

        X[province].append(x)
        Y[province].append(y)


In [139]:
from sklearn.preprocessing import MinMaxScaler

x_scalers, y_scalers = {}, {}

X_scaled = defaultdict(lambda: list())
Y_scaled = defaultdict(lambda: list())

for province in provinces:
    x = data[province].drop(['PM2.5'], axis=1)
    y = data[province][['PM2.5']]

    x_scaler = MinMaxScaler().fit(x)
    y_scaler = MinMaxScaler().fit(y)

    for e in X[province]:
        X_scaled[province].append(x_scaler.transform(e))
    for e in Y[province]:
        Y_scaled[province].append(y_scaler.transform(e))

In [140]:
X['Bangkok'][0]

,Temp(C),WindDir,Wind Speed(km/h),Cambodia_frp,Myanmar_frp,Thailand_frp,Lao_PDR_frp
2016-03-03 23:00:00,27.3,70.0,15.0,1.574234,1.238528,4.12,1.274845
2016-03-04 00:00:00,27.3,70.0,15.0,1.574234,1.238528,4.12,1.274845


In [141]:
Y['Bangkok'][0]

,PM2.5
2016-03-04 01:00:00,43.1
2016-03-04 02:00:00,44.8
2016-03-04 03:00:00,37.5
2016-03-04 04:00:00,39.6
2016-03-04 05:00:00,46.0
...,...
2016-03-06 20:00:00,26.0
2016-03-06 21:00:00,32.4
2016-03-06 22:00:00,32.8
2016-03-06 23:00:00,24.8


## Reshaping input

In [145]:
bkk_x = np.array(X_scaled['Bangkok'])
bkk_y = np.array(Y_scaled['Bangkok'])
print(f'X : {bkk_x.shape}')
print(f'Y : {bkk_y.shape}')

X : (4424, 2, 7)
Y : (4424, 72, 1)


In [146]:
n_features = len(data[province].columns)-1
x_train = bkk_x.reshape((bkk_x.shape[0], timesteps, n_features))
y_train = bkk_y.reshape((bkk_y.shape[0], 72))

In [147]:
print(f'x : {x_train.shape}')
print(f'y : {y_train.shape}')

x : (4424, 2, 7)
y : (4424, 72)


## Building Model

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf

# define model
model = Sequential([
    LSTM(100, activation='relu', input_shape=(n_input, n_features)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(generator,epochs=5, batch_size=100)

Test

In [31]:
df.head(4)

,Temp,WindDir,Wind Speed,PM2.5
2016-03-03 07:00:00,26.4,65,15,62.900000
2016-03-03 10:00:00,31.4,75,13,52.966667
2016-03-03 13:00:00,34.1,70,12,35.266667
2016-03-03 16:00:00,34.6,60,12,33.050000


In [25]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [28]:

# define generator
n_input = 2921
n_features = 1
generator = TimeseriesGenerator(x, y, length=2, batch_size=1)

In [33]:
X, Y = generator[1]

In [34]:
X

array([[[31.4, 75. , 13. ],
        [34.1, 70. , 12. ]]])

In [35]:
Y

array([33.05])